In [ ]:
import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
        print("Setting GPU Memory Growth...")
        tf.config.experimental.set_memory_growth(physical_devices[0], True)
        
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

import time
import datetime

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from plot_keras_history import plot_history
import sklearn.metrics as metrics
import os as os

import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GRU
from tensorflow.python.client import device_lib

import kerastuner as kt
from kerastuner.tuners import RandomSearch, BayesianOptimization


print("TF version:", tf.__version__)
print("Panda version:", pd.__version__)

def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

print("Physical devices: ", tf.config.list_physical_devices())
# Se va a habilitar la dedicacion dinamica de memoria para que la GPU vaya asignando recursos al proceso conforme los vaya necesitando

if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print("Build with CUDA: ", tf.test.is_built_with_cuda())

# Insantes de tiempo futuros a predecir
numPredictions = 600

# Porcentaje del conjunto de test
test_size = 0.3
# Porcentaje del conjunto de validacion
val_size = 0.3

# Establecer objetivo (Name of model metric to minimize or maximize, e.g. "val_accuracy"). el "val_" hace referencia a que se coge la métrica en el subconjunto de validación

# obje = kerastuner.Objective('val_mean_absolute_percentage_error', 'min')

# Epocas
epchs=1000
# Tamaño del batch
batch = 1024
# Dimensionalidad del espacio de output
units = 16

# Establecer medida de loss
loss="mean_squared_error"

# Model metrics
modelMetrics = [keras.metrics.MAE, keras.metrics.RMSE]

# Establecer learning rate
lr = [0.0, 1e-2, 1e-3, 1e-4]

In [ ]:
df = pd.read_excel("C:/Users/hecto/Documents/Master/TFM/tfm-renewable-energy-deep-learning/data/2015_1min.xlsx", 
                    header=None,
                    engine='openpyxl')[0]

df2 = pd.read_excel("C:/Users/hecto/Documents/Master/TFM/tfm-renewable-energy-deep-learning/data/2016_1min.xlsx",
                    header = None, 
                    engine = 'openpyxl')[0]

In [ ]:
# Data preprocessing: each row will contain the 600 measures for each day , and the 600 measures for the following day
X = pd.DataFrame(np.array(df).reshape(-1, numPredictions))
Y = pd.DataFrame.copy(X)

Y.columns = ["col_{}".format(i) for i in range(601, 1201)]
Y = Y.drop(0)
Y = Y.reset_index(drop=True)
Y.loc[len(Y)] = np.zeros(numPredictions)

# Last row is deleted because it is the one used for the real prediction, 
# it is not useful for the training of the model. 
X.drop(X.tail(1).index,inplace=True)
Y.drop(Y.tail(1).index,inplace=True)

print("X Preproccessed shape: ", X.shape)
print("Y Preproccessed shape: ", Y.shape)
print("---------------------------------------------")

#  Uncomment in order to normalize data
# # Data Normalization
# scaler = MinMaxScaler()
# XNormalized = pd.DataFrame(scaler.fit_transform(X), columns=["col_{}".format(i) for i in range(1, 601)])
# YNormalized = pd.DataFrame(scaler.fit_transform(Y), columns=["col_{}".format(i) for i in range(601, 1201)])

dfPreproccessed = pd.concat([X, Y], axis=1)

print("DataFrame Preproccessed:")
print(dfPreproccessed)
print("---------------------------------------------")

# Split the data into training and validation sets
xTrain, xTest, yTrain, yTest = train_test_split(X, 
                                                Y, 
                                                test_size = test_size, 
                                                random_state = 0, 
                                                shuffle=False)
xTrain, xVal, yTrain, yVal = train_test_split(xTrain, 
                                            yTrain, 
                                            test_size = val_size, 
                                            random_state = 0,
                                            shuffle=False)


In [ ]:
with tf.device('/gpu:0'):
    def build_model(hp):

        # Model definition
        model = keras.models.Sequential()
        model.add(GRU(
                units = units ,
                input_shape = (xTrain.shape[1], 1),
            ))
        model.add(Dropout(rate = 0.25))
        model.add(Dense(numPredictions, activation="sigmoid"))

        print("Model Summary: ", model.summary())

        # Set Epsilon to 1, in order to fix huge MAPE values.
        keras.backend.set_epsilon(1)

        # Model compilation
        model.compile(loss = loss,
                    optimizer = "adam",
                    metrics = modelMetrics)

        history = model.fit(
            np.reshape(xTrain.values, (xTrain.shape[0], xTrain.shape[1], 1)),
            yTrain,
            epochs=epchs,
            batch_size=batch,
            validation_data=(np.reshape(xVal.values, (xVal.shape[0], xVal.shape[1], 1)), yVal))
    

In [ ]:
# Training and Validation loss curves
plt.plot(history.history['loss'], label='Training loss')
plt.plot(history.history['val_loss'], label='Validation loss')
plt.legend()
plt.show()

# Model evaluation with validation data
score = model.evaluate(np.reshape(xVal.values, (xVal.shape[0], xVal.shape[1], 1)), yVal)
print('Score:', score)